In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import pandas as pd
import re
from PIL import Image

In [ ]:
source_dir = '/content/drive/MyDrive/Processing parameters'
destination_dir = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)


In [ ]:
def rename_and_move_files(source, destination):
    print(f"Listing directories in source: {source}")  # Debugging print
    for folder in os.listdir(source):
        print(f"Processing folder: {folder}")  # Debugging print
        folder_path = os.path.join(source, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                new_filename = f"{folder}_{file}"
                src_file_path = os.path.join(folder_path, file)
                dest_file_path = os.path.join(destination, new_filename)
                shutil.copy2(src_file_path, dest_file_path)
                print(f"Moved: {src_file_path} to {dest_file_path}")
        else:
            print(f"Skipped (not a directory): {folder_path}")  # Debugging print for non-directory entries

# Execute the function
rename_and_move_files(source_dir, destination_dir)

In [ ]:
import os
import pandas as pd
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to extract metadata from image names and folders
def extract_metadata(folder, image_name):
    metadata = {
        'Folder': folder,
        'Image Name': image_name,
        'Spincoating Speed': '',
        'Temperature (°C)': '',
        'Composition': '',
        'Antisolvent Used': ''
    }

    if folder == 'Antisolvent':
        prefix = image_name.split('_')[0]
        metadata['Spincoating Speed'] = '1000rpm 2s, 6000rpm 30s'
        metadata['Temperature (°C)'] = 110
        metadata['Composition'] = 'AgBiI4'
        antisolvent_mapping = {
            'CB': 'chlorobenzene',
            'Tol': 'toluene',
            'IPA': 'isopropanol',
            'none': 'none used'
        }
        metadata['Antisolvent Used'] = antisolvent_mapping.get(prefix, '')

    elif folder == 'Composition':
        composition_mapping = {
            'AgBiI4': ('6000rpm 30s', 'RT', 'AgBiI4'),
            'Ag2BiI5': ('6000rpm 30s', 'RT', 'Ag2BiI5'),
            'Ag3BiI6': ('6000rpm 30s', 'RT', 'Ag3BiI6')
        }
        for comp, details in composition_mapping.items():
            if comp in image_name:
                metadata['Spincoating Speed'], metadata['Temperature (°C)'], metadata['Composition'] = details
                metadata['Antisolvent Used'] = 'none used'
                break

    elif folder == 'Spinspeed 4000 vs 6000':
        if '4000' in image_name:
            metadata['Spincoating Speed'] = '1000rpm 2s, 4000rpm 30s'
        elif '6000' in image_name:
            metadata['Spincoating Speed'] = '1000rpm 2s, 6000rpm 30s'
        metadata['Temperature (°C)'] = 110
        metadata['Composition'] = 'AgBiI4'
        metadata['Antisolvent Used'] = 'isopropanol' if 'IPA' in image_name else 'none used'

    elif folder in ['Temp 50-150', 'Temp 100-150']:
        temperature = int(''.join(re.findall(r'\d+', image_name)))  # Extracts continuous digits
        metadata['Temperature (°C)'] = temperature
        metadata['Spincoating Speed'] = '6000rpm 30s' if folder == 'Temp 50-150' else '1000rpm 2s, 6000rpm 30s'
        metadata['Composition'] = 'AgBiI4' if folder == 'Temp 50-150' else 'AgBiI5'
        metadata['Antisolvent Used'] = 'none used'

    return metadata

# Function to populate the DataFrame with metadata
def populate_metadata_df(source_dir):
    columns = ['Folder', 'Image Name', 'Spincoating Speed', 'Temperature (°C)', 'Composition', 'Antisolvent Used']
    metadata_records = []

    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                if image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp','.tif')):
                    metadata = extract_metadata(folder, image_name)
                    metadata_records.append(metadata)

    return pd.DataFrame(metadata_records, columns=columns)

# Main script to generate the metadata DataFrame
source_dir = '/content/drive/MyDrive/Processing parameters'  # Adjust path as needed
metadata_df = populate_metadata_df(source_dir)

# Save the metadata DataFrame to a CSV file
metadata_csv_path = '/content/drive/MyDrive/DAISY2.0 Data/metadata_dec_data.csv'  # Adjust path as needed
metadata_df.to_csv(metadata_csv_path, index=False)

print("Metadata and bright particles analysis completed. Results saved to:", metadata_csv_path)


In [ ]:
import os
from PIL import Image

# Define the folder paths
input_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data'
output_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped'
os.makedirs(output_folder, exist_ok=True)

# Define the percentage of the image to crop
remove_percentage = 0.10  # Crop the bottom 10% of the image

# Process each file in the folder
file_list = os.listdir(input_folder)
for file_name in file_list:
    # Check the file extension
    if file_name.lower().endswith(('.tif', '.png', '.jpg')):
        # Open the image
        image_path = os.path.join(input_folder, file_name)
        image = Image.open(image_path)

        # Get image dimensions
        width, height = image.size

        # Calculate the height to keep (top 90%)
        keep_height = int(height * (1 - remove_percentage))

        # Calculate the cropping region (top 90%)
        crop_region = (0, 0, width, keep_height)

        # Crop the image
        cropped_image = image.crop(crop_region)

        # Define the filename for the cropped image, keeping the extension as .tif
        tif_filename = os.path.splitext(file_name)[0] + '.tif'
        tif_file_path = os.path.join(output_folder, tif_filename)

        # Save the cropped image in TIFF format
        cropped_image.save(tif_file_path, 'TIFF')

print("Cropping complete. All images are saved as .tif in the output folder.")
